In [2]:
import PyQt5
print(PyQt5.__file__)




/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.8/site-packages/PyQt5/__init__.py


In [1]:
import sys
from PyQt5.QtWidgets import (QApplication, QMainWindow, QSlider, QWidget, QVBoxLayout,
                             QHBoxLayout, QLabel, QLineEdit, QComboBox, QPushButton,
                             QGridLayout, QFrame, QSizePolicy)  # Added QSizePolicy here
from PyQt5.QtCore import (Qt, QThread, pyqtSignal)
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import numpy as np

from realStock import get_realtime_stock_price
from realOption import main as get_realtime_option_price

class FetchStockThread(QThread):
    # Define a signal to send the fetched data back to the main thread
    data_fetched = pyqtSignal(float)

    def __init__(self, stock_name):
        super().__init__()
        self.stock_name = stock_name

    def run(self):
        price = get_realtime_stock_price(self.stock_name)
        self.data_fetched.emit(price)

class FetchOptionThread(QThread):
    data_fetched = pyqtSignal(list)

    def __init__(self, company, date, strike):
        super().__init__()
        self.company = company
        self.date = date
        self.strike = strike

    def run(self):
        prices = get_realtime_option_price(self.company, self.date, self.strike)
        self.data_fetched.emit(prices)


stylesheet = """
QWidget {
    font-family: Verdana, Arial, Helvetica, sans-serif;
    font-size: 14px;
    color: #333;
}

QComboBox {
    border: 1px solid #ccc;
    border-radius: 4px;
    background: white;
    text-align: center; /* Center text in QComboBox */
    height: 32px;
    padding: 24px;
    font-size: 22px;
}

QComboBox::drop-down {
    subcontrol-origin: padding;
    subcontrol-position: top right;
    width: 15px;
    border-left-width: 1px;
    border-left-color: darkgray;
    border-left-style: solid; /* just a single line */
    border-top-right-radius: 3px; /* same radius as the QComboBox */
    border-bottom-right-radius: 3px;
}

QComboBox QAbstractItemView {
    selection-background-color: #ccc;
    text-align: center; /* Center text in the dropdown items */
}

QLineEdit {
    border: 1px solid #ccc;
    padding: 5px;
    border-radius: 4px;
    background: white;
    font-size: 14px;
    text-align: center;
}

QLabel {
    font-size: 14px;
}

QSlider::groove:horizontal {
    border: 1px solid #999;
    height: 8px;
    border-radius: 4px;
    background: qlineargradient(x1:0, y1:0, x2:0, y2:1, stop:0 #eee, stop:1 #ddd);
}

QSlider::handle:horizontal {
    background: white;
    border: 1px solid #ccc;
    width: 18px;
    margin: -2px 0;
    border-radius: 3px;
}

QPushButton {
    border: 1px solid #ccc;
    border-radius: 4px;
    padding: 5px 10px;
    background: #eee;
}

QPushButton:pressed {
    background: #ddd;
    font-size: 16px;
}

QPushButton:hover {
    border-color: #bbb;
}
"""


class OptionStrategyVisualizer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.setStyleSheet(stylesheet)

    def initUI(self):
        
        self.setWindowTitle("Option Strategy Visualizer")
        self.setGeometry(100, 100, 1357, 768)  # Set to the image width and height

        # Central widget and layout
        central_widget = QWidget(self)
        self.setCentralWidget(central_widget)
        grid_layout = QGridLayout(central_widget)

        # Left-side controls
        control_panel = QFrame(central_widget)
        control_layout = QVBoxLayout(control_panel)

        # Adjustments for control panel layout and dimensions
        control_panel.setMaximumWidth(300)  # Adjust this value to make the control panel narrower

        # Trade Type Combo Box
        self.trade_type_combo = QComboBox(control_panel)
        self.trade_type_combo.addItems(['Buy Call-Buy Put', 'Buy Call-Sell Put', 'Sell Call-Buy Put', 'Sell Call-Sell Put'])
        control_layout.addWidget(self.trade_type_combo)

        # Sliders
        self.nCall_slider = self.create_slider('NCalls', 0, 5, 1) 
        control_layout.addWidget(self.nCall_slider)

        self.nPut_slider = self.create_slider('NPuts', 0, 5, 1)
        control_layout.addWidget(self.nPut_slider)

        self.deltaCall_slider = self.create_slider('DeltaCall', 0, 1, 0.01) 
        control_layout.addWidget(self.deltaCall_slider)

        self.deltaPut_slider = self.create_slider('DeltaPut', -1, 0, 0.01)
        control_layout.addWidget(self.deltaPut_slider)

        # Input Fields
        # Adjust the layout for the input fields to be in lines
        input_layout_1 = QHBoxLayout()
        self.symbol_input = self.create_input_field('Symbol', 'AAPL')  # Default symbol
        input_layout_1.addWidget(self.symbol_input)
        
        input_layout_2 = QHBoxLayout()
        self.date_input = self.create_input_field('Maturity_Date', 'YYYY-MM-DD')  # Default date
        input_layout_2.addWidget(self.date_input)
        
        input_layout_3 = QHBoxLayout()
        self.x_input = self.create_input_field('Strike_Price', '150')
        input_layout_3.addWidget(self.x_input)

        self.fetch_data_button = QPushButton('Fetch Data / Refresh', control_panel)
        self.fetch_data_button.clicked.connect(self.fetch_data)  # Connect button click to the fetch_data method
           
        input_layout_4 = QHBoxLayout()
        self.call_premium_input = self.create_input_field('C', '9.8')
        self.put_premium_input = self.create_input_field('P', '14.5')
        input_layout_4.addWidget(self.call_premium_input)
        input_layout_4.addWidget(self.put_premium_input)

        input_layout_5 = QHBoxLayout()
        self.stock_price_input = self.create_input_field('SPrice', '150')
        self.stock_range_input = self.create_input_field('SRange', '0.05')
        input_layout_5.addWidget(self.stock_price_input)
        input_layout_5.addWidget(self.stock_range_input)

        input_layout_6 = QHBoxLayout()
        self.y_min_input = self.create_input_field('Y_Min', '-500')
        self.y_max_input = self.create_input_field('Y_Max', '500')
        input_layout_6.addWidget(self.y_min_input)
        input_layout_6.addWidget(self.y_max_input)

        # Add input layouts to the main control layout
        control_layout.addLayout(input_layout_1)
        control_layout.addLayout(input_layout_2)
        control_layout.addLayout(input_layout_3)
        control_layout.addWidget(self.fetch_data_button)
        control_layout.addLayout(input_layout_4)
        control_layout.addLayout(input_layout_5)
        control_layout.addLayout(input_layout_6)

        # Add control panel to grid
        grid_layout.addWidget(control_panel, 0, 0)

        # Right-side plot
        self.figure = Figure()
        self.canvas = FigureCanvas(self.figure)
        self.canvas.setMinimumWidth(950)  # Increase this value to make the plot area wider
        grid_layout.addWidget(self.canvas, 0, 1)
        
        self.toolbar = NavigationToolbar(self.canvas, self)
        grid_layout.addWidget(self.toolbar, 1, 1)  # Place it under the canvas or wherever you see fit

        # Adjust the column stretch factors to change the proportions
        grid_layout.setColumnStretch(0, 35)  # Less stretch for controls column
        grid_layout.setColumnStretch(1, 65)  # More stretch for plot column

        # Connect signals to update method
        self.nCall_slider.slider.valueChanged.connect(self.update_plot)
        self.nPut_slider.slider.valueChanged.connect(self.update_plot)
        self.deltaCall_slider.slider.valueChanged.connect(self.update_plot)
        self.deltaPut_slider.slider.valueChanged.connect(self.update_plot)
        self.x_input.input_field.textChanged.connect(self.update_plot)
        self.call_premium_input.input_field.textChanged.connect(self.update_plot)
        self.put_premium_input.input_field.textChanged.connect(self.update_plot)
        self.stock_price_input.input_field.textChanged.connect(self.update_plot)
        self.stock_range_input.input_field.textChanged.connect(self.update_plot)
        self.y_min_input.input_field.textChanged.connect(self.update_plot)
        self.y_max_input.input_field.textChanged.connect(self.update_plot)

        # Show the window
        self.show()

    def create_slider(self, label, min_val, max_val, precision):
        # Use a QWidget to hold the layout
        container = QWidget()
        layout = QHBoxLayout()
        lbl = QLabel(label)
        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(int(min_val / precision))
        slider.setMaximum(int(max_val / precision))
        slider.valueChanged.connect(lambda value: lbl.setText(f'{label}: {value * precision}'))
        slider.valueChanged.connect(self.update_plot)  # Connect the slider to the update_plot method
        layout.addWidget(lbl)
        layout.addWidget(slider)
        container.setLayout(layout)
        container.slider = slider  # Store the slider in the container for access
        return container

    def create_input_field(self, label, default_value):
        container = QWidget()
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)  # Keep margins minimal
        layout.setSpacing(10)
        
        lbl = QLabel(label)
        lbl.setSizePolicy(QSizePolicy.MinimumExpanding, QSizePolicy.Fixed)  # Adjust this line
        
        input_field = QLineEdit(default_value)
        input_field.setAlignment(Qt.AlignCenter)
        input_field.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Fixed)  # Ensure input field can expand
        
        layout.addWidget(lbl)
        layout.addWidget(input_field)
        container.setLayout(layout)
        container.input_field = input_field
        return container



    def update_plot(self):
        # Retrieve values from UI components
        n_call = self.nCall_slider.slider.value()
        n_put = self.nPut_slider.slider.value()
        delta_call = self.deltaCall_slider.slider.value() / 100.0
        delta_put = self.deltaPut_slider.slider.value() / 100.0
        stock_price = float(self.stock_price_input.input_field.text())
        X = float(self.x_input.input_field.text())
        call_premium = float(self.call_premium_input.input_field.text())
        put_premium = float(self.put_premium_input.input_field.text())
        stock_range = float(self.stock_range_input.input_field.text())
        Y_min = float(self.y_min_input.input_field.text())
        Y_max = float(self.y_max_input.input_field.text())

        # Calculate values
        S_min = np.floor(stock_price * (1 - stock_range))
        S_max = np.ceil(stock_price * (1 + stock_range))
        S_grid = np.arange(S_min, S_max + 1)
        call_at_maturity = np.maximum(S_grid - X, 0)
        put_at_maturity = np.maximum(X - S_grid, 0)

        # Logic for plotting based on trade type
        trade_type = self.trade_type_combo.currentText()
        if trade_type == 'Buy Call-Buy Put':
            y_option = n_call * (call_at_maturity - call_premium) + n_put * (put_at_maturity - put_premium)
            y_stock = n_call * delta_call * (S_grid - stock_price) + n_put * delta_put * (S_grid - stock_price)
            effective_delta = -n_call * delta_call - n_put * delta_put
        if trade_type == 'Buy Call-Sell Put':
            y_option = n_call * (call_at_maturity - call_premium) - n_put * (put_at_maturity - put_premium)
            y_stock = n_call * delta_call * (S_grid - stock_price) - n_put * delta_put * (S_grid - stock_price)
            effective_delta = -n_call * delta_call + n_put * delta_put
        if trade_type == 'Sell Call-Buy Put':
            y_option = -n_call * (call_at_maturity - call_premium) + n_put * (put_at_maturity - put_premium)
            y_stock = -n_call * delta_call * (S_grid - stock_price) + n_put * delta_put * (S_grid - stock_price)
            effective_delta = n_call * delta_call - n_put * delta_put
        if trade_type == 'Sell Call-Sell Put': 
            y_option = -n_call * (call_at_maturity - call_premium) - n_put * (put_at_maturity - put_premium)
            y_stock = -n_call * delta_call * (S_grid - stock_price) - n_put * delta_put * (S_grid - stock_price)
            effective_delta = n_call * delta_call + n_put * delta_put

        # Plotting logic
        y = 100 * (y_option + y_stock)
        yp = np.maximum(y, 0)
        y_neg = np.where(yp == 0)[0]
        y_pos = np.where(yp > 0)[0]

        if y_neg.size != 0 and y_pos.size != 0:
            if y_neg[0] > 0 and y_neg[-1] < len(S_grid) - 1:
                pos_str = f'(0, {S_grid[y_neg[0] - 1]}) and ({S_grid[y_neg[-1] + 1]}, ∞)'
            else:
                pos_range = S_grid[y_pos]
                if y_pos[0] == 0:
                    pos_str = f'(0, {pos_range[-1]})'
                else:
                    pos_str = f'({pos_range[0]}, {pos_range[-1]})'
        elif y_neg.size == 0:
            pos_str = '(0, ∞)'
        else:
            pos_str = '∞'

        title_str = [f'100Δ_effective = {round(100 * effective_delta)}. Make money if S ∈ {pos_str}',
                     f'n_call = {n_call}, n_put = {n_put}, Δ_call = {delta_call:.2f}, Δ_put = {delta_put:.2f}']

        
        # Inside your update_plot method, before plotting:
        with plt.style.context('ggplot'):
            self.figure.clear()
            ax = self.figure.add_subplot(111)

            # Plot with a thicker line and use a different color for visibility
            ax.plot(S_grid, y, 'blue', linewidth=2.5, label='Strategy Profit/Loss')

            # Add a fill between
            ax.fill_between(S_grid, y, where=(y > 0), color='green', alpha=0.3, label='Profit')
            ax.fill_between(S_grid, y, where=(y <= 0), color='red', alpha=0.3, label='Loss')

            # Improve title and label aesthetics
            ax.set_title("\n".join(title_str), fontsize=16, fontweight='bold')
            ax.set_xlabel('Stock Price', fontsize=14)
            ax.set_ylabel('Pay-off at Maturity', fontsize=14)

            # Add gridlines
            ax.grid(True, which='major', linestyle='--', linewidth='0.5', color='black')

            # Add legend
            ax.legend()

            ax.axhline(0, color='k', linewidth=2)
            ax.set_ylim([Y_min, Y_max])
            ax.set_xlim([S_min, S_max])

            self.canvas.draw()
    
    def update_stock_price_input(self, price):
        formatted_price = "{:.2f}".format(price)
        self.stock_price_input.input_field.setText(formatted_price)

 
    def fetch_data(self):
        # Check if symbol, strike_price, and maturity_date fields are filled
        if self.symbol_input.input_field.text() and self.x_input.input_field.text() and self.date_input.input_field.text():
            company = self.symbol_input.input_field.text()
            date = self.date_input.input_field.text()
            strike = float(self.x_input.input_field.text())

            # Initiate thread to fetch stock price
            self.fetch_stock_price(company)

            # Initiate thread to fetch option premiums
            self.update_option_premiums()
        else:
            print("Please ensure all fields (symbol, strike price, and maturity date) are filled.")
        
    def fetch_stock_price(self, company):
        """Initiates fetching of the stock price for the given company symbol."""
        # Ensure any existing thread is terminated before starting a new one
        if hasattr(self, 'stock_fetch_thread'):
            self.stock_fetch_thread.quit()
            self.stock_fetch_thread.wait()

        self.stock_fetch_thread = FetchStockThread(company)
        self.stock_fetch_thread.data_fetched.connect(self.update_stock_price_input)
        self.stock_fetch_thread.start()

        
    def update_option_premiums(self):
        company = self.symbol_input.input_field.text()
        date = self.date_input.input_field.text()
        strike = float(self.x_input.input_field.text())

        # Cancel any existing thread to avoid overlapping requests
        if hasattr(self, 'option_fetch_thread'):
            self.option_fetch_thread.terminate()

        self.option_fetch_thread = FetchOptionThread(company, date, strike)
        self.option_fetch_thread.data_fetched.connect(self.fill_premium_inputs)
        self.option_fetch_thread.start()

    def fill_premium_inputs(self, prices):
        if prices:
            self.call_premium_input.input_field.setText(str(prices[0]))
            self.put_premium_input.input_field.setText(str(prices[1]))

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = OptionStrategyVisualizer()
    sys.exit(app.exec_())

Call option(s) for strike price 150.0 on 2024-04-19: AAPL240419C00150000
Put option(s) for strike price 150.0 on 2024-04-19: AAPL240419P00150000
['AAPL240419C00150000', 'AAPL240419P00150000']
Current Option is AAPL240419C00150000
The market is closed, the close price of AAPL is 171.3700
Market is closed. The last recorded transaction price of AAPL240419C00150000 was 22.27.
Current Option is AAPL240419P00150000
Market is closed. The last recorded transaction price of AAPL240419P00150000 was 0.31.


SystemExit: 0

/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


$a + b$

In [2]:
a, b = 5, 2
a + b

7

In [1]:
from scipy.stats import norm
import numpy as np

class BlackScholes:
    
    def N(self, x):
        return norm.cdf(x)
    
    def blsprice(self, cp_flag, S, X, T, r, v):
        d1 = (np.log(S / X) + (r + 0.5 * v ** 2) * T) / (v * np.sqrt(T))
        d2 = d1 - v * np.sqrt(T)
        if cp_flag == 'c':
            price = S * self.N(d1) - X * np.exp(-r * T) * self.N(d2)
        else:
            price = X * np.exp(-r * T) * self.N(-d2) - S * self.N(-d1)
        return price
    
    def blsdelta(self, cp_flag, S, X, T, r, v):
        d1 = (np.log(S / X) + (r + 0.5 * v ** 2) * T) / (v * np.sqrt(T))
        if cp_flag == 'c':
            delta = self.N(d1)
        else:
            delta = self.N(d1) - 1
        return delta
    
    def blsvega(self, cp_flag, S, X, T, r, v):
        d1 = (np.log(S / X) + (r + 0.5 * v ** 2) * T) / (v * np.sqrt(T))
        vega = S * np.sqrt(T) * norm.pdf(d1)
        return vega

    def blsimpv(self, cp_flag, S, X, T, r, C, sigma, tol=1e-6, max_iterations=100):
        for i in range(max_iterations):
            price = self.blsprice(cp_flag, S, X, T, r, sigma)
            vega = self.blsvega(cp_flag, S, X, T, r, sigma)
            
            if vega == 0:
                return np.nan
            
            diff = C - price
            if abs(diff) < tol:
                return sigma
            sigma = sigma + diff / vega
            if sigma <= 0:  # sigma must be positive, if not, return an error
                return np.nan
        
        return np.nan  # if no convergence, return NaN
    
# Initialize BlackScholes class
bs_model = BlackScholes()

# Known inputs
S = 170.42  # Current AMD stock price
X = 120  # Strike price
T = 12/365 * 365  # Time to expiration in years (assuming 1 year for simplicity)
r = 0.01  # Risk-free interest rate
v = 0.2  # Volatility
C = 9.8  # Market call price (example)
P = 14.5  # Market put price (example)

# Calculate theoretical call and put prices
call_price = bs_model.blsprice('c', S, X, T, r, v)
put_price = bs_model.blsprice('p', S, X, T, r, v)

# Calculate implied volatility (using call price as an example, assuming initial sigma as 0.2)
impv_call = bs_model.blsimpv('c', S, X, T, r, C, v)
impv_put = bs_model.blsimpv('p', S, X, T, r, P, v)

# Calculate deltas
call_delta = bs_model.blsdelta('c', S, X, T, r, v)
put_delta = bs_model.blsdelta('p', S, X, T, r, v)

(call_price, put_price, impv_call, impv_put, call_delta, put_delta)


(77.33571656724928,
 13.346168973308188,
 nan,
 0.20823734764128626,
 0.8475339952418088,
 -0.15246600475819116)

In [19]:
def getIndexOption(symbol, ticker):
    info = yf.Ticker(symbol)
    option_syb = ticker[:next((i for i, char in enumerate(ticker) if char.isdigit()), None)]
    length = len(option_syb)
    date = ticker[length:length + 6]
    option_date = f"20{date[:2]}-{date[2:4]}-{date[4:]}"
    opt = info.option_chain(option_date)
    optionType = ticker[length + 6]
    if optionType.upper() == "C":
        calls = opt.calls
        res = calls[calls['contractSymbol'] == ticker]
    else:
        puts = opt.puts
        res = puts[puts['contractSymbol'] == ticker]
    
    last = res['lastPrice'].values[0]
    bid = res['bid'].values[0]
    ask = res['ask'].values[0]
    return last, bid, ask
getIndexOption('^NDX', 'NDXP240722C18200000')

(1535.3, 1642.9, 1677.8)

In [15]:
import yfinance as yf
ticker = yf.Ticker('^SPX')
ticker.history(period='1d')
ticker.option_chain('2024-08-16')

opt = ticker.option_chain('2024-08-16')
calls = opt.calls
puts = opt.puts

# get contract symbol 'SPX240816C04500000
calls[calls['contractSymbol'] == 'SPX240816C04500000']['ask'].values[0]

1087.4

In [15]:
# get the stock price, price change, and percentage change
import yfinance as yf

ticker = 'AAPL'
stock = yf.Ticker(ticker)
# stock.info['regularMarketPrice'], stock.info['regularMarketChange'], stock.info['regularMarketChangePercent']
previous_close = stock.info['regularMarketPreviousClose']
current_price = 
price_change = current_price - previous_close
percent_change = round((price_change / previous_close * 100), 2)
current_price, price_change, percent_change


KeyError: 'regularMarketPrice'

In [16]:
import yfinance as yf

ticker = 'AAPL'
stock = yf.Ticker(ticker)
stock.info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod

In [1]:
import yfinance as yf

# Define the ETF ticker
ticker = "^XSP"

# Fetch data
etf = yf.Ticker(ticker)

# Get the current price
current_price = etf.history(period="1d")['Close'].iloc[0]

print(f"The current price of {ticker} is ${current_price:.2f}")


The current price of ^XSP is $505.00


In [5]:
from realOption import main as get_realtime_option_price

get_realtime_option_price('XSP', '20240425', '455')

No options available for 20240425.
[]


[]

In [2]:
import yfinance as yf
from datetime import datetime
import holidays
import pytz

def get_realtime_stock_price(stock_name):
    stock = yf.Ticker(stock_name)
    today = datetime.today()
    
    # Check for weekends and holidays
    if today.weekday() > 4 or today in holidays.UnitedStates(years=today.year):
        todays_data = stock.history(period="1d")
        if not todays_data.empty:
            current_price = todays_data['Close'].iloc[-1]
            previous_close = stock.info.get('regularMarketPreviousClose', current_price)
            price_change = current_price - previous_close
            percent_change = round((price_change / previous_close) * 100, 2)
            status = "closed"
        else:
            return None  # No data available
    else:
        try:
            # Try getting current price during market hours
            current_price = stock.info.get("currentPrice")
            previous_close = stock.info.get('regularMarketPreviousClose', current_price)
            if current_price is None:
                # Fallback if currentPrice is not available
                todays_data = stock.history(period="1d")
                if not todays_data.empty:
                    current_price = todays_data['Close'].iloc[-1]
            price_change = current_price - previous_close
            percent_change = round((price_change / previous_close) * 100, 2)
            status = "open"
        except Exception as e:
            print(f"Error retrieving stock data: {e}")
            return None

    print(f"The market is {status}, the current price of {stock_name} is {current_price:.2f}, the price change is {price_change:.2f}, the percent change is {percent_change:.2f}%")
    return current_price, price_change, percent_change

# Example usage
get_realtime_stock_price("^SPX")


The market is open, the current price of ^SPX is 5038.07, the price change is 19.68, the percent change is 0.39%


(5038.06982421875, 19.679824218749673, 0.39)

In [1]:
import requests
from bs4 import BeautifulSoup

def fetch_option_data(option_chain):
    url = f"https://finance.yahoo.com/quote/{option_chain}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Example: Finding the price element by inspecting the webpage
        # Note: These selectors are hypothetical and need to be adjusted based on actual page structure
        price = soup.find('fin-streamer', {'data-field': 'regularMarketPrice'}).text
        change = soup.find('fin-streamer', {'data-field': 'regularMarketChange'}).text
        percent_change = soup.find('fin-streamer', {'data-field': 'regularMarketChangePercent'}).text
        
        return price, change, percent_change
    else:
        print("Failed to retrieve data")
        return None

# Example usage

option_data = fetch_option_data('ADBE240719P00415000')
if option_data:
    print(option_data)


('0.4900', '0.0000', '(0.00%)')


In [51]:
spx = yf.Ticker("^SPX")
spx.options
# 2024-05-02 info
spx.option_chain('2024-05-02')
# strike price 4500 calls and puts
call = spx.option_chain('2024-05-02').calls[spx.option_chain('2024-05-02').calls['strike'] == 4500]
call_symbol = spx.option_chain('2024-05-02').calls[spx.option_chain('2024-05-02').calls['strike'] == 4500]['contractSymbol'].values[0]
put_symbol = spx.option_chain('2024-05-02').puts[spx.option_chain('2024-05-02').puts['strike'] == 4500]['contractSymbol'].values[0]

call_price = spx.option_chain('2024-05-02').calls[spx.option_chain('2024-05-02').calls['strike'] == 4500]['lastPrice'].values[0]
call_open_interest = spx.option_chain('2024-05-02').calls[spx.option_chain('2024-05-02').calls['strike'] == 4500]['openInterest'].values[0]
call_volume = spx.option_chain('2024-05-02').calls[spx.option_chain('2024-05-02').calls['strike'] == 4500]['volume'].values[0]
call_symbol, call_price, call_open_interest, call_volume

('SPXW240502C04500000', 549.69, 8, 8.0)

In [57]:
spx = yf.Ticker("^SPX")
spx.options
# 2024-05-02 info
spx.option_chain('2024-05-02')
# strike price 4500 calls and puts
call = spx.option_chain('2024-05-02').calls[spx.option_chain('2024-05-02').calls['strike'] == 4500]
call_symbol = spx.option_chain('2024-05-02').calls[spx.option_chain('2024-05-02').calls['strike'] == 4500]['contractSymbol'].values[0]
call_symbol[:next((i for i, char in enumerate(call_symbol) if char.isdigit()), None)]

'SPXW'

In [4]:
import requests
import datetime

api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
symbol = 'AAPL'
expiration_date = '2024-06-21'
strike_price = 180.0
option_type = 'P'  # 'C' for call, 'P' for put

# Construct the URL
url = f'https://api.polygon.io/v3/reference/options/contracts'

params = {
    'underlying_ticker': symbol,
    'expiration_date': expiration_date,
    'strike_price': strike_price,
    'type': option_type,
    'apiKey': api_key
}

response = requests.get(url, params=params)

if response.status_code == 200:
    try:
        data = response.json()
        results = data.get('results', [])
        if results:
            for option in results:
                print(f"Ticker: {option['ticker']}")
                print(f"Expiration Date: {option['expiration_date']}")
                print(f"Strike Price: {option['strike_price']}")
        else:
            print("No options data found.")
    except requests.exceptions.JSONDecodeError as e:
        print("Error decoding JSON response")
        print(f"Response content: {response.text}")
else:
    print(f"Error: Received status code {response.status_code}")
    print(f"Response content: {response.text}")


No options data found.


In [9]:
url = 'https://api.polygon.io/v3/snapshot/options/EVRI?limit=10&apiKey=C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
response = requests.get(url)
response.json()


{'status': 'NOT_AUTHORIZED',
 'request_id': 'ba97fd964f73d8ab9db47eafeeee3764',
 'message': 'You are not entitled to this data. Please upgrade your plan at https://polygon.io/pricing'}

In [13]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'

base_url = 'https://api.polygon.io/v2/aggs/ticker/'

def get_options_close_price(ticker, date):
    expiration_date = datetime(2024, 5, 17)
    start_date = expiration_date - timedelta(days=100)
    end_date = datetime.now()

    url = f"{base_url}{ticker}/range/1/day/{start_date.strftime('%Y-%m-%d')}/{end_date.strftime('%Y-%m-%d')}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['results'])
        print(df)
        df['t'] = pd.to_datetime(df['t'], unit='ms')
        df['t'] = df['t'].dt.date

        date_obj = datetime.strptime(date, '%Y-%m-%d').date()
        if date_obj in df['t'].values:
            close_price = df[df['t'] == date_obj]['c'].values[0]
            return close_price
        else:
            print(f"No data available for the date: {date}")
            return None
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

# Example usage:
ticker = 'O:SPXW240702C04000000'
for date in pd.date_range('2024-05-14', '2024-05-15', freq='B').strftime('%Y-%m-%d'):
    close_price = get_options_close_price(ticker, date)
    # set a time delay to avoid hitting the API rate limit
    time.sleep(1)
    if close_price is not None:
        print(f"Close price on {date}: {close_price}")



   v       vw        o        c        h        l              t  n
0  2  1430.33  1430.33  1430.33  1430.33  1430.33  1718251200000  2
No data available for the date: 2024-05-14
   v       vw        o        c        h        l              t  n
0  2  1430.33  1430.33  1430.33  1430.33  1430.33  1718251200000  2
No data available for the date: 2024-05-15


In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
from realPrice.realStock import get_realtime_stock_price
from realPrice.realOption import main as get_realtime_option_price

def get_historical_data(ticker, start_date):
    api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
    base_url = 'https://api.polygon.io/v2/aggs/ticker/'
    end_date = datetime.now()  

    ticker = f'O:{ticker}'
    if isinstance(start_date, datetime):
        start_date = start_date.strftime('%Y-%m-%d')

    url = f"{base_url}{ticker}/range/1/day/{start_date}/{end_date.strftime('%Y-%m-%d')}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df['date'] = df['t'].dt.date  
            return df[['date','c',]]
        else:
            print("No results found in the data.")
            return pd.DataFrame()  
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None


def calls_or_puts(company, date, strike):
    options = [] 
    ticker = yf.Ticker(company)
    expiration_dates = ticker.options

    if date in expiration_dates:
        opts = ticker.option_chain(date)
        
        call_option = opts.calls[opts.calls['strike'] == strike]
        put_option = opts.puts[opts.puts['strike'] == strike]
        
        if not call_option.empty:
            call_option_names = call_option['contractSymbol'].tolist()
            options.extend(call_option_names)
            print(f"Call option(s) for strike price {strike} on {date}: {', '.join(call_option_names)}")
        else:
            print(f"No call option with a strike price of {strike} for {date}.")
            
        if not put_option.empty:
            put_option_names = put_option['contractSymbol'].tolist()
            options.extend(put_option_names)
            print(f"Put option(s) for strike price {strike} on {date}: {', '.join(put_option_names)}")
        else:
            print(f"No put option with a strike price of {strike} for {date}.")
    else:
        print(f"No options available for {date}.")
    return options

def get_stock_price(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    hist.reset_index(inplace=True)
    hist['date'] = hist['Date'].dt.date
    # rename "Close" to 'stock_close_price'
    hist.rename(columns={'Close': 'stock_close_price'}, inplace=True)
    # let all stock to two digits
    hist['stock'] = hist['stock_close_price'].round(2)
    return hist[['date', 'stock_close_price']]

def main(company='TSM', strike_date='2024-06-21', strike=160, trade_date='2024-05-16'):
    options = calls_or_puts(company, strike_date, strike)
    if options:
        data_frames = []
        for i, option in enumerate(options):
            price_data = get_historical_data(option, trade_date)
            time.sleep(1)
            if price_data is not None:
                if i == 0:
                    price_data.rename(columns={'c': 'call_close_price'}, inplace=True)
                else:    
                    price_data.rename(columns={'c': 'put_close_price'}, inplace=True)
                data_frames.append(price_data)
            else:
                print(f"Failed to retrieve data for option: {option}")

        if len(data_frames) == 2:
            df = pd.merge(data_frames[0], data_frames[1], on='date')
            
            # also get the stock date
            start_date = datetime.strptime(trade_date, '%Y-%m-%d')
            end_date = datetime.now()
            stock_prices = get_stock_price(company, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
            df = pd.merge(df, stock_prices, on='date', how='left')
            print(df)
            
            # above only get info till yesterday, get today's stock price, call price and put price
            current_price = get_realtime_stock_price(company)[0]
            current_call_price = get_realtime_option_price(company, strike_date, strike)[0]
            current_put_price = get_realtime_option_price(company, strike_date, strike)[1]
            
            # add today's stock price, call price and put price to the dataframe
            df.loc[len(df)] = [datetime.now().date(), current_call_price, current_put_price, current_price]
            
            return df
        else:
            print("Could not retrieve data for one or more options.")
            return None
    else:
        print("No options found.")
        return None

print(main())



ModuleNotFoundError: No module named 'realPrice'

In [23]:
get_historical_data('AAPL240621C00180000', '2024-05-20')

Failed to retrieve data: 404


In [3]:
import yfinance as yf
def get_stock_price(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    hist.reset_index(inplace=True)
    hist['date'] = hist['Date'].dt.date
    # rename "Close" to 'stock_close_price'
    hist.rename(columns={'Close': 'stock_close_price'}, inplace=True)
    # let all stock to two digits
    hist['stock'] = hist['stock_close_price'].round(2)
    return hist[['date', 'stock_close_price']]
get_stock_price('SPY', '2024-05-20', '2024-05-21')

,date,stock_close_price
0,2024-05-20,528.355469


In [1]:
import importlib.metadata

def get_installed_packages():
    # Retrieve installed packages along with their versions
    packages = {dist.metadata['Name']: dist.version for dist in importlib.metadata.distributions()}
    return packages

if __name__ == "__main__":
    installed_packages = get_installed_packages()
    for name, version in installed_packages.items():
        print(f"{name}=={version}")


wheel==0.42.0
kiwisolver==1.4.5
wrds==3.1.6
contourpy==1.1.1
pexpect==4.9.0
multitasking==0.0.9
decorator==5.1.1
tzdata==2024.1
mplcursors==0.5.3
nest_asyncio==1.6.0
python-dateutil==2.9.0
termcolor==2.4.0
ptyprocess==0.7.0
outcome==1.3.0.post0
altair-viewer==0.4.0
unicodedata2==15.1.0
portpicker==1.6.0
toml==0.10.2
Pygments==2.17.2
psycopg2-binary==2.9.9
webencodings==0.5.1
yfinance==0.2.37
pyqtgraph==0.13.3
altair-data-server==0.4.1
importlib_metadata==7.0.2
yaspin==2.2.0
ply==3.11
referencing==0.35.1
selenium==4.21.0
pickleshare==0.7.5
stack-data==0.6.2
ipython==8.12.2
holidays==0.45
plotly==5.22.0
jsonschema-specifications==2023.12.1
parso==0.8.3
six==1.16.0
seaborn==0.13.2
altair-saver==0.5.0
h11==0.14.0
jupyter_core==5.7.1
attrs==23.2.0
widgetsnbextension==4.0.10
Jinja2==3.1.3
pytz==2024.1
PySocks==1.7.1
statsmodels==0.14.1
scipy==1.10.1
appnope==0.1.4
trio==0.25.1
packaging==23.2
appdirs==1.4.4
pyparsing==3.1.2
pooch==1.8.1
metatrader==0.0.1
comm==0.2.1
SQLAlchemy==1.4.52
platfo

In [15]:
import yfinance as yf
from realPrice import get_realtime_stock_price

def get_option_chain(company='SPX', date='2024-05-02', strike=4500):
    ticker = yf.Ticker(company)
    option_chain = ticker.option_chain(date)
   
    call_data = option_chain.calls[option_chain.calls['strike'] == strike]
    
    if not call_data.empty:
        call_symbol = call_data['contractSymbol'].values[0]
        call_symbol = call_symbol[:next((i for i, char in enumerate(call_symbol) if char.isdigit()), None)]
        return call_symbol
    else:
        return None
    
def get_historical_data(ticker, start_date):
    api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
    base_url = 'https://api.polygon.io/v2/aggs/ticker/'
    end_date = datetime.now()  

    ticker = f'O:{ticker}'
    if isinstance(start_date, datetime):
        start_date = start_date.strftime('%Y-%m-%d')

    url = f"{base_url}{ticker}/range/1/day/{start_date}/{end_date.strftime('%Y-%m-%d')}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df['date'] = df['t'].dt.date  
            return df[['date','c',]]
        else:
            print("No results found in the data.")
            return pd.DataFrame()  
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def get_stock_price(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    hist.reset_index(inplace=True)
    hist['date'] = hist['Date'].dt.date
    hist.rename(columns={'Close': 'stock_close_price'}, inplace=True)
    hist['stock_close_price'] = hist['stock_close_price'].round(2)
    return hist[['date', 'stock_close_price']]
    
def main(company='SPX', date='2024-05-02', strike=4500, trade_date='2024-05-16'):
    # Fetch ticker information
    ticker = yf.Ticker(company)
    option_chain = ticker.option_chain(date)
  
    
    calls = option_chain.calls
    puts = option_chain.puts
    call_data = calls[calls['strike'] == strike]
    put_data = puts[puts['strike'] == strike]

    if call_data.empty and put_data.empty:
        print("No call or put options available for the specified strike price.")
        return None
    
    options = []
    realPrices = []
    
    if not call_data.empty:
        call_symbol = call_data['contractSymbol'].values[0]
        options.append(call_symbol)
        call_price = call_data['lastPrice'].values[0]
        realPrices.append(call_price)
    
    if not put_data.empty:
        put_symbol = put_data['contractSymbol'].values[0]
        options.append(put_symbol)
        put_price = put_data['lastPrice'].values[0]
        realPrices.append(put_price)

    # Get historical data for the options
    data_frames = []
    for i, option in enumerate(options):
        price_data = get_historical_data(option, trade_date)
        time.sleep(1)
        if price_data is not None:
            if i == 0:
                price_data.rename(columns={'c': 'call_close_price'}, inplace=True)
            else:    
                price_data.rename(columns={'c': 'put_close_price'}, inplace=True)
            data_frames.append(price_data)
        else:
            print(f"Failed to retrieve data for option: {option}")
    
    # Merge the dataframes
    if len(data_frames) == 2:
        df = pd.merge(data_frames[0], data_frames[1], on='date')
        # Get the stock price data
        start_date = datetime.strptime(trade_date, '%Y-%m-%d')
        end_date = datetime.now()
        stock_prices = get_stock_price('SPY', start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
        df = pd.merge(df, stock_prices, on='date', how='left')
        
        # add real time stock price, call price as realPrices[0], put price as realPrices[1]
        current_price = get_realtime_stock_price('SPY')[0]
        call_price = realPrices[0]
        put_price = realPrices[1]
        
        df.loc[len(df)] = [datetime.now().date(), call_price, put_price, current_price]
        
        print(df)
        return df
    else:
        print("Could not retrieve data for one or more options.")
        return None
    
  






SyntaxError: invalid syntax (2541969170.py, line 2)

In [14]:
main('^SPX', '2024-07-19', 4500)

Option Symbol: ['SPXW240719C04500000', 'SPX240719P04500000']


[[981.7, 0.55], [], []]

In [1]:
import yfinance as yf
def calls_and_puts(company='SPX', date='2024-05-02', strike=4500):
    ticker = yf.Ticker(company)
    option_chain = ticker.option_chain(date)
  
    
    calls = option_chain.calls
    puts = option_chain.puts
    call_data = calls[calls['strike'] == strike]
    put_data = puts[puts['strike'] == strike]
    options = []
    call_symbol = call_data['contractSymbol'].values[0]
    options.append(call_symbol)
    put_symbol = put_data['contractSymbol'].values[0]
    options.append(put_symbol)  
calls_and_puts('^SPX', '2024-07-19', 4500)    

In [13]:
def get_historical_data(ticker, start_date):
    api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
    base_url = 'https://api.polygon.io/v2/aggs/ticker/'
    end_date = datetime.now()  

    ticker = f'O:{ticker}'
    if isinstance(start_date, datetime):
        start_date = start_date.strftime('%Y-%m-%d')

    url = f"{base_url}{ticker}/range/1/day/{start_date}/{end_date.strftime('%Y-%m-%d')}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df['date'] = df['t'].dt.date  
            return df[['date','c',]]
        else:
            print("No results found in the data.")
            return pd.DataFrame()  
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None
get_historical_data('ADBE240816C00470000', '2024-06-12'), get_historical_data('ADBE240816P00470000', '2024-06-12')


(          date       c
 0   2024-06-12   25.54
 1   2024-06-13   24.45
 2   2024-06-14   63.55
 3   2024-06-17   51.00
 4   2024-06-18   63.50
 5   2024-06-20   61.99
 6   2024-06-21   69.85
 7   2024-06-24   63.60
 8   2024-06-25   62.55
 9   2024-06-27   82.15
 10  2024-06-28   90.00
 11  2024-07-01   87.20
 12  2024-07-08  102.86
 13  2024-07-09   98.45
 14  2024-07-10   93.86
 15  2024-07-11   93.93
 16  2024-07-15   97.99,
           date      c
 0   2024-06-12  23.90
 1   2024-06-13  31.75
 2   2024-06-14   3.75
 3   2024-06-17   4.30
 4   2024-06-18   3.48
 5   2024-06-20   3.50
 6   2024-06-21   2.18
 7   2024-06-24   2.65
 8   2024-06-25   2.85
 9   2024-06-26   2.32
 10  2024-06-27   1.57
 11  2024-06-28   1.16
 12  2024-07-01   1.02
 13  2024-07-02   1.06
 14  2024-07-05   0.75
 15  2024-07-08   0.68
 16  2024-07-10   0.81
 17  2024-07-11   0.71
 18  2024-07-12   0.60
 19  2024-07-15   0.41
 20  2024-07-16   0.41)

In [41]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
import numpy as np
import holidays

project_dir = '/Users/julienne_hu/Desktop/Option_Apps'
sys.path.append(project_dir)

from realPrice.realStock import get_realtime_stock_price
from realPrice.realOption import main as get_realtime_option_price

def get_historical_data(ticker, start_date):
    api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
    base_url = 'https://api.polygon.io/v2/aggs/ticker/'
    end_date = datetime.now()  

    ticker = f'O:{ticker}'
    if isinstance(start_date, datetime):
        start_date = start_date.strftime('%Y-%m-%d')

    url = f"{base_url}{ticker}/range/1/day/{start_date}/{end_date.strftime('%Y-%m-%d')}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df['date'] = df['t'].dt.date  
            return df[['date', 'c']]
        else:
            print("No results found in the data.")
            return pd.DataFrame()  
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def calls_or_puts(company, date, strike):
    options = [] 
    ticker = yf.Ticker(company)
    expiration_dates = ticker.options

    if date in expiration_dates:
        opts = ticker.option_chain(date)
        
        call_option = opts.calls[opts.calls['strike'] == strike]
        put_option = opts.puts[opts.puts['strike'] == strike]
        
        if not call_option.empty:
            call_option_names = call_option['contractSymbol'].tolist()
            options.extend(call_option_names)
            print(f"Call option(s) for strike price {strike} on {date}: {', '.join(call_option_names)}")
        else:
            print(f"No call option with a strike price of {strike} for {date}.")
            
        if not put_option.empty:
            put_option_names = put_option['contractSymbol'].tolist()
            options.extend(put_option_names)
            print(f"Put option(s) for strike price {strike} on {date}: {', '.join(put_option_names)}")
        else:
            print(f"No put option with a strike price of {strike} for {date}.")
    else:
        print(f"No options available for {date}.")
    return options

def get_stock_price(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    hist.reset_index(inplace=True)
    hist['date'] = hist['Date'].dt.date
    hist.rename(columns={'Close': 'stock_close_price'}, inplace=True)
    hist['stock_close_price'] = hist['stock_close_price'].round(2)
    return hist[['date', 'stock_close_price']]

def initialize_df(trade_date):
    df = pd.DataFrame(columns=['date', 'call_close_price', 'put_close_price', 'stock_close_price'])
    
    today = datetime.now().date()
    df['date'] = pd.date_range(trade_date, today).strftime('%Y-%m-%d')
    df['date'] = pd.to_datetime(df['date'])

    df['call_close_price'] = np.nan
    df['put_close_price'] = np.nan
    df['stock_close_price'] = np.nan

 
    df = df[df['date'].dt.weekday < 5]

    us_holidays = holidays.US(years=[2024])
    df = df[~df['date'].isin(us_holidays)]
    
    return df

def main(company='ADBE', strike_date='2024-08-16', strike=470, trade_date='2024-06-12'):
    options = calls_or_puts(company, strike_date, strike)
    if options:
        data_frames = []
        for i, option in enumerate(options):
            price_data = get_historical_data(option, trade_date)
            time.sleep(1)
            if price_data is not None:
                if i == 0:
                    price_data.rename(columns={'c': 'call_close_price'}, inplace=True)
                else:    
                    price_data.rename(columns={'c': 'put_close_price'}, inplace=True)
                data_frames.append(price_data)
            else:
                print(f"Failed to retrieve data for option: {option}")

        if len(data_frames) == 2:
            # merge the dataframes keep all the data
            df = pd.merge(data_frames[0], data_frames[1], on='date', how='outer')
            df['date'] = pd.to_datetime(df['date'])
            # sort data by date
            df = df.sort_values(by='date')
       
            # create a list to have all the dates from trade_date to today excluding weekends and holidays
            trade_date = '2024-06-12'
            today = datetime.now().date()-timedelta(days=1)
            us_holidays = holidays.US(years=[2024])
            allday = pd.date_range(trade_date, today).strftime('%Y-%m-%d')
            allday = [day for day in allday if pd.to_datetime(day).weekday() < 5 and day not in us_holidays ]

            # add dates that are not in the df but in the allday list
            missing_dates = [day for day in allday if day not in df['date'].dt.strftime('%Y-%m-%d').values]
            missing_df = pd.DataFrame(missing_dates, columns=['date'])
            missing_df['date'] = pd.to_datetime(missing_df['date'])
            missing_df['call_close_price'] = np.nan
            missing_df['put_close_price'] = np.nan
            df = pd.concat([df, missing_df], ignore_index=True)
            df = df.sort_values(by='date')

            # Get the stock price data
            start_date = datetime.strptime(trade_date, '%Y-%m-%d')
            end_date = datetime.now()
            stock_prices = get_stock_price(company, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
            
            # Ensure 'date' column in stock_prices is datetime format
            stock_prices['date'] = pd.to_datetime(stock_prices['date'])

            df = pd.merge(df, stock_prices, on='date', how='left')
            df['date'] = df['date'].dt.date
            
            # Get today's stock price, call price, and put price
            current_price = get_realtime_stock_price(company)[0]
            current_call_price = get_realtime_option_price(company, strike_date, strike)[0]
            current_put_price = get_realtime_option_price(company, strike_date, strike)[1]
            
            # Add today's data to the DataFrame if it's not already present else replace it
            df.loc[len(df)] = [datetime.now().date(), current_call_price, current_put_price, current_price]
            
            # for nan values, fill them with the previous day's value
            df = df.fillna(method='ffill')
            return df
        else:
            print("Could not retrieve data for one or more options.")
            return None
    else:
        print("No options found.")
        return None

result_df = main()
print(result_df)





Call option(s) for strike price 470 on 2024-08-16: ADBE240816C00470000
Put option(s) for strike price 470 on 2024-08-16: ADBE240816P00470000
The current price of ADBE is 563.09, the price change is -3.45, the percent change is -0.61%
Call option(s) for strike price 470 on 2024-08-16: ADBE240816C00470000
Put option(s) for strike price 470 on 2024-08-16: ADBE240816P00470000
Current Option is ADBE240816C00470000
Last price: 97.99, Ask: 101.0, Bid: 92.0.
Current Option is ADBE240816P00470000
Last price: 0.6, Ask: 0.91, Bid: 0.32.
Call option(s) for strike price 470 on 2024-08-16: ADBE240816C00470000
Put option(s) for strike price 470 on 2024-08-16: ADBE240816P00470000
Current Option is ADBE240816C00470000
Last price: 97.99, Ask: 101.0, Bid: 92.0.
Current Option is ADBE240816P00470000
Last price: 0.6, Ask: 0.91, Bid: 0.32.
          date  call_close_price  put_close_price  stock_close_price
0   2024-06-12             25.54            23.90             459.87
1   2024-06-13             24.45

In [25]:
import pandas as pd
from datetime import datetime
import numpy as np
import holidays

# Create a DataFrame with columns date, call_close_price, put_close_price, stock_close_price
df = pd.DataFrame(columns=['date', 'call_close_price', 'put_close_price', 'stock_close_price'])

# Define trade_date and today
trade_date = '2024-06-12'
today = datetime.now().date()

# Create a date range from trade_date to today
df['date'] = pd.date_range(trade_date, today).strftime('%Y-%m-%d')
df['date'] = pd.to_datetime(df['date'])

# Initialize call_close_price, put_close_price, stock_close_price with NaN
df['call_close_price'] = np.nan
df['put_close_price'] = np.nan
df['stock_close_price'] = np.nan

# Remove weekends
df = df[df['date'].dt.weekday < 5]

# Remove US holidays
us_holidays = holidays.US(years=[2024])
df = df[~df['date'].isin(us_holidays)]


print(df)


         date  call_close_price  put_close_price  stock_close_price
0  2024-06-12               NaN              NaN                NaN
1  2024-06-13               NaN              NaN                NaN
2  2024-06-14               NaN              NaN                NaN
3  2024-06-17               NaN              NaN                NaN
4  2024-06-18               NaN              NaN                NaN
5  2024-06-20               NaN              NaN                NaN
6  2024-06-21               NaN              NaN                NaN
7  2024-06-24               NaN              NaN                NaN
8  2024-06-25               NaN              NaN                NaN
9  2024-06-26               NaN              NaN                NaN
10 2024-06-27               NaN              NaN                NaN
11 2024-06-28               NaN              NaN                NaN
12 2024-07-01               NaN              NaN                NaN
13 2024-07-02               NaN              NaN

In [25]:
trade_date = '2024-06-12'
today = datetime.now().date()
us_holidays = holidays.US(years=[2024])
allday = pd.date_range(trade_date, today).strftime('%Y-%m-%d')
allday = [day for day in allday if pd.to_datetime(day).weekday() < 5 and day not in us_holidays ]

allday

['2024-06-12',
 '2024-06-13',
 '2024-06-14',
 '2024-06-17',
 '2024-06-18',
 '2024-06-20',
 '2024-06-21',
 '2024-06-24',
 '2024-06-25',
 '2024-06-26',
 '2024-06-27',
 '2024-06-28',
 '2024-07-01',
 '2024-07-02',
 '2024-07-03',
 '2024-07-05',
 '2024-07-08',
 '2024-07-09',
 '2024-07-10',
 '2024-07-11',
 '2024-07-12',
 '2024-07-15',
 '2024-07-16',
 '2024-07-17']

In [5]:
import sys

project_dir = '/Users/julienne_hu/Desktop/Option_Apps'
sys.path.append(project_dir)

import yfinance as yf
from realPrice.realStock import get_realtime_stock_price
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import numpy as np
import holidays


def get_option_chain(company='SPX', date='2024-05-02', strike=4500):
    ticker = yf.Ticker(company)
    option_chain = ticker.option_chain(date)
   
    call_data = option_chain.calls[option_chain.calls['strike'] == strike]
    
    if not call_data.empty:
        call_symbol = call_data['contractSymbol'].values[0]
        call_symbol = call_symbol[:next((i for i, char in enumerate(call_symbol) if char.isdigit()), None)]
        return call_symbol
    else:
        return None
    
def get_historical_data(ticker, start_date):
    api_key = 'C6ig1sXku2yKl_XEIvSvc_OWCwB8ILLn'
    base_url = 'https://api.polygon.io/v2/aggs/ticker/'
    end_date = datetime.now()  

    ticker = f'O:{ticker}'
    if isinstance(start_date, datetime):
        start_date = start_date.strftime('%Y-%m-%d')

    url = f"{base_url}{ticker}/range/1/day/{start_date}/{end_date.strftime('%Y-%m-%d')}?apiKey={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df['date'] = df['t'].dt.date  
            return df[['date','c',]]
        else:
            print("No results found in the data.")
            return pd.DataFrame()  
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

def get_stock_price(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    hist.reset_index(inplace=True)
    hist['date'] = hist['Date'].dt.date
    hist.rename(columns={'Close': 'stock_close_price'}, inplace=True)
    hist['stock_close_price'] = hist['stock_close_price'].round(2)
    return hist[['date', 'stock_close_price']]

def calls_and_puts(company='SPX', date='2024-05-02', strike=4500):
    ticker = yf.Ticker(company)
    option_chain = ticker.option_chain(date)
  
    
    calls = option_chain.calls
    puts = option_chain.puts
    call_data = calls[calls['strike'] == strike]
    put_data = puts[puts['strike'] == strike]
    options = []
    call_symbol = call_data['contractSymbol'].values[0]
    options.append(call_symbol)
    put_symbol = put_data['contractSymbol'].values[0]
    options.append(put_symbol) 
    return options 
    
def main(company='^SPX', date='2024-08-16', strike=4700, trade_date='2024-07-01'):
    # Fetch ticker information
    ticker = yf.Ticker(company)
    option_chain = ticker.option_chain(date)
  
    
    calls = option_chain.calls
    puts = option_chain.puts
    call_data = calls[calls['strike'] == strike]
    put_data = puts[puts['strike'] == strike]

    if call_data.empty and put_data.empty:
        print("No call or put options available for the specified strike price.")
        return None
    
    options = []
    realPrices = []
    
    if not call_data.empty:
        call_symbol = call_data['contractSymbol'].values[0]
        options.append(call_symbol)
        call_price = call_data['lastPrice'].values[0]
        realPrices.append(call_price)
    
    if not put_data.empty:
        put_symbol = put_data['contractSymbol'].values[0]
        options.append(put_symbol)
        put_price = put_data['lastPrice'].values[0]
        realPrices.append(put_price)

    # Get historical data for the options
    data_frames = []
    for i, option in enumerate(options):
        price_data = get_historical_data(option, trade_date)
        time.sleep(1)
        if price_data is not None:
            if i == 0:
                price_data.rename(columns={'c': 'call_close_price'}, inplace=True)
            else:    
                price_data.rename(columns={'c': 'put_close_price'}, inplace=True)
            data_frames.append(price_data)
        else:
            print(f"Failed to retrieve data for option: {option}")
    
    # Merge the dataframes
    if len(data_frames) == 2:
        df = pd.merge(data_frames[0], data_frames[1], on='date')
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values(by='date')
        
        today = datetime.now().date()-timedelta(days=1)
        us_holidays = holidays.US(years=[2024])
        allday = pd.date_range(trade_date, today).strftime('%Y-%m-%d')
        allday = [day for day in allday if pd.to_datetime(day).weekday() < 5 and day not in us_holidays ]

        # add dates that are not in the df but in the allday list
        missing_dates = [day for day in allday if day not in df['date'].dt.strftime('%Y-%m-%d').values]
        missing_df = pd.DataFrame(missing_dates, columns=['date'])
        missing_df['date'] = pd.to_datetime(missing_df['date'])
        missing_df['call_close_price'] = np.nan
        missing_df['put_close_price'] = np.nan
        df = pd.concat([df, missing_df], ignore_index=True)
        df = df.sort_values(by='date')
        
        # Get the stock price data
        start_date = datetime.strptime(trade_date, '%Y-%m-%d')
        end_date = datetime.now()
        stock_prices = get_stock_price('SPY', start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
        stock_prices['date'] = pd.to_datetime(stock_prices['date'])
        df = pd.merge(df, stock_prices, on='date', how='left')
        df['date'] = df['date'].dt.date
            
        # add real time stock price, call price as realPrices[0], put price as realPrices[1]
        current_price = get_realtime_stock_price('SPY')[0]
        call_price = realPrices[0]
        put_price = realPrices[1]
        
        df.loc[len(df)] = [datetime.now().date(), call_price, put_price, current_price]
        df = df.fillna(method='ffill')
        # round the stock price to two digits
        df['stock_close_price'] = df['stock_close_price'].round(2)
        return df
    else:
        print("Could not retrieve data for one or more options.")
        return None
    
main()




The current price of SPY is 556.94, the price change is -7.92, the percent change is -1.40%
          date  call_close_price  put_close_price  stock_close_price
0   2024-07-01               NaN              NaN             545.34
1   2024-07-02            833.15             3.40             549.01
2   2024-07-03            857.73             3.20             551.46
3   2024-07-05            857.73             3.20             554.64
4   2024-07-08            898.00             2.54             555.28
5   2024-07-09            898.00             2.54             555.82
6   2024-07-10            937.63             2.45             561.32
7   2024-07-11            937.63             2.45             556.48
8   2024-07-12            958.08             2.07             559.99
9   2024-07-15            958.08             2.07             561.53
10  2024-07-16            958.08             2.07             564.86
11  2024-07-17            958.08             2.07             556.94
12  2024-07

,date,call_close_price,put_close_price,stock_close_price
0,2024-07-01,NaN,NaN,545.34
1,2024-07-02,833.15,3.40,549.01
2,2024-07-03,857.73,3.20,551.46
3,2024-07-05,857.73,3.20,554.64
4,2024-07-08,898.00,2.54,555.28
5,2024-07-09,898.00,2.54,555.82
6,2024-07-10,937.63,2.45,561.32
7,2024-07-11,937.63,2.45,556.48
8,2024-07-12,958.08,2.07,559.99
9,2024-07-15,958.08,2.07,561.53


In [9]:
import decimal

decimal.Decimal('1').exp()

Decimal('2.718281828459045235360287471')

In [20]:
from mpmath import *
import math

mp.dps = 64; mp.pretty = True

# Value from mpmath
e1 = mpmathify(math.exp(1))

# Value from wolframalpha.com
e2 = 2.7182818284590452353602874713526624977572470936999595749669676277
# Error
print(e1)

2.718281828459045090795598298427648842334747314453125


In [15]:
len('2.141592653589793238462643383279502884197')

41

In [27]:
import math
math_Pi = math.pi
wa_Pi = 3.1415926535897932384626433832795028841971693993751058209749445923

In [32]:
from mpmath import *
mp.dps = 65
mp_Pi = pi() 
fsub(mp_Pi, wa_Pi, prec=65)

0.00000000000000012246467991473531772014792699320386414420353931996486690470492265

In [29]:
len('0.0000000000000001')

20

In [1]:
from scipy import zeros

zero_vector = zeros(10)

print(zero_vector)


ImportError: cannot import name 'zeros' from 'scipy' (/opt/homebrew/lib/python3.11/site-packages/scipy/__init__.py)

In [2]:
import scipy

print(scipy.__version__)


1.13.0
